### Geocoder putting longitude and latitude 

Now that we have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood, but first lets restablish the data we got from part 1 of the capstone. 

In [1]:
import numpy as np

In [2]:
import pandas as pd

In [3]:
from bs4 import BeautifulSoup

In [4]:
import geocoder

In [5]:
import requests

In [6]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text

In [7]:
soup = BeautifulSoup(source, 'xml')

In [8]:
table=soup.find('table')

In [9]:
column_names = ['Postalcode','Borough','Neighbourhood']
df = pd.DataFrame(columns = column_names)

In [10]:
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [11]:
df=df[df['Borough']!='Not assigned']

In [12]:
temp_df=df.groupby('Postalcode')['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x))
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighbourhood':'Neighbourhood_joined'},inplace=True)

In [13]:
df_merge = pd.merge(df, temp_df, on='Postalcode')

In [14]:
df_merge.drop_duplicates(inplace=True)

In [15]:
df_merge.rename(columns={'Neighbourhood_joined':'Neighbourhood'},inplace=True)

In [16]:
lat_lng_coords = None

In [17]:
#Using the provided geocode function
def geocode(postal_code):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return longitude, latitude

In [18]:
geo_df=pd.read_csv('http://cocl.us/Geospatial_data')

In [20]:
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [21]:
geo_df.rename(columns={'Postal Code':'Postalcode'},inplace=True)

In [22]:
geo_merged = pd.merge(geo_df, df_merge, on='Postalcode')

In [23]:
geo_data=geo_merged[['Postalcode','Borough','Neighbourhood','Latitude','Longitude']]

In [26]:
geo_data.head(12)

,Postalcode,Borough,Neighbourhood,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge","Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek","Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill","Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park","Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge","Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West","Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West","Birch Cliff, Cliffside West",43.692657,-79.264848


Part 2 Done